In [2]:
!pip install torch==1.0.0 torchvision==0.2.1
!pip install Pillow==6.2.2

     |████████████████████████████████| 591.8MB 28kB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.0.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


     |████████████████████████████████| 2.1MB 11.1MB/s 
ERROR: bokeh 2.3.2 has requirement pillow>=7.1.0, but you'll have pillow 6.2.2 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.0.0
True


In [2]:
!python3 -m pip install SSIM-PIL

In [3]:
# https://github.com/yjn870/SRCNN-pytorch
import argparse
import glob
import h5py
import numpy as np
import PIL.Image as pil_image
from SSIM_PIL import compare_ssim
import torch
import numpy as np

# utils
def convert_rgb_to_y(img):
    if type(img) == np.ndarray:
        return 16. + (64.738 * img[:, :, 0] + 129.057 * img[:, :, 1] + 25.064 * img[:, :, 2]) / 256.
    elif type(img) == torch.Tensor:
        if len(img.shape) == 4:
            img = img.squeeze(0)
        return 16. + (64.738 * img[0, :, :] + 129.057 * img[1, :, :] + 25.064 * img[2, :, :]) / 256.
    else:
        raise Exception('Unknown Type', type(img))


def convert_rgb_to_ycbcr(img):
    if type(img) == np.ndarray:
        y = 16. + (64.738 * img[:, :, 0] + 129.057 * img[:, :, 1] + 25.064 * img[:, :, 2]) / 256.
        cb = 128. + (-37.945 * img[:, :, 0] - 74.494 * img[:, :, 1] + 112.439 * img[:, :, 2]) / 256.
        cr = 128. + (112.439 * img[:, :, 0] - 94.154 * img[:, :, 1] - 18.285 * img[:, :, 2]) / 256.
        return np.array([y, cb, cr]).transpose([1, 2, 0])
    elif type(img) == torch.Tensor:
        if len(img.shape) == 4:
            img = img.squeeze(0)
        y = 16. + (64.738 * img[0, :, :] + 129.057 * img[1, :, :] + 25.064 * img[2, :, :]) / 256.
        cb = 128. + (-37.945 * img[0, :, :] - 74.494 * img[1, :, :] + 112.439 * img[2, :, :]) / 256.
        cr = 128. + (112.439 * img[0, :, :] - 94.154 * img[1, :, :] - 18.285 * img[2, :, :]) / 256.
        return torch.cat([y, cb, cr], 0).permute(1, 2, 0)
    else:
        raise Exception('Unknown Type', type(img))


def convert_ycbcr_to_rgb(img):
    if type(img) == np.ndarray:
        r = 298.082 * img[:, :, 0] / 256. + 408.583 * img[:, :, 2] / 256. - 222.921
        g = 298.082 * img[:, :, 0] / 256. - 100.291 * img[:, :, 1] / 256. - 208.120 * img[:, :, 2] / 256. + 135.576
        b = 298.082 * img[:, :, 0] / 256. + 516.412 * img[:, :, 1] / 256. - 276.836
        return np.array([r, g, b]).transpose([1, 2, 0])
    elif type(img) == torch.Tensor:
        if len(img.shape) == 4:
            img = img.squeeze(0)
        r = 298.082 * img[0, :, :] / 256. + 408.583 * img[2, :, :] / 256. - 222.921
        g = 298.082 * img[0, :, :] / 256. - 100.291 * img[1, :, :] / 256. - 208.120 * img[2, :, :] / 256. + 135.576
        b = 298.082 * img[0, :, :] / 256. + 516.412 * img[1, :, :] / 256. - 276.836
        return torch.cat([r, g, b], 0).permute(1, 2, 0)
    else:
        raise Exception('Unknown Type', type(img))


def calc_psnr(img1, img2):
    return 10. * torch.log10(1. / torch.mean((img1 - img2) ** 2))

def calc_ssim(img1, img2):
    return compare_ssim(img1, img2)


class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# generate dataset
def trainset_generate(images_dir, output_path, scale, stride=56, patch_size=66):
    h5_file = h5py.File(output_path, 'w')

    lr_patches = []
    hr_patches = []
    index = 1

    for image_path in sorted(glob.glob('{}/*'.format(images_dir))):
        print('process {0} image'.format(index))
        hr = pil_image.open(image_path).convert('RGB')
        hr_width = (hr.width // scale) * scale
        hr_height = (hr.height // scale) * scale
        hr = hr.resize((hr_width, hr_height), resample=pil_image.BICUBIC)
        lr = hr.resize((hr_width // scale, hr_height // scale), resample=pil_image.BICUBIC)
        lr = lr.resize((lr.width * scale, lr.height * scale), resample=pil_image.BICUBIC)
        hr = np.array(hr).astype(np.float32)
        lr = np.array(lr).astype(np.float32)
        hr = convert_rgb_to_y(hr)
        lr = convert_rgb_to_y(lr)

        for i in range(0, lr.shape[0] - patch_size + 1, stride):
            for j in range(0, lr.shape[1] - patch_size + 1, stride):
                lr_patches.append(lr[i:i + patch_size, j:j + patch_size])
                hr_patches.append(hr[i:i + patch_size, j:j + patch_size])
        index = index + 1

    lr_patches = np.array(lr_patches)
    hr_patches = np.array(hr_patches)

    h5_file.create_dataset('lr', data=lr_patches)
    h5_file.create_dataset('hr', data=hr_patches)

    h5_file.close()


# generate training dataset
scale = 2
trainset_generate('/content/drive/MyDrive/CS17 project/data/HR/', '/content/drive/MyDrive/CS17 project/data/train.hdf5', scale=scale)

process 1 image
process 2 image
process 3 image
process 4 image
process 5 image
process 6 image
process 7 image
process 8 image
process 9 image
process 10 image
process 11 image
process 12 image
process 13 image
process 14 image
process 15 image
process 16 image
process 17 image
process 18 image
process 19 image
process 20 image
process 21 image
process 22 image
process 23 image
process 24 image
process 25 image
process 26 image
process 27 image
process 28 image
process 29 image
process 30 image
process 31 image
process 32 image
process 33 image
process 34 image
process 35 image
process 36 image
process 37 image
process 38 image
process 39 image
process 40 image
process 41 image
process 42 image
process 43 image
process 44 image
process 45 image
process 46 image
process 47 image
process 48 image
process 49 image
process 50 image
process 51 image
process 52 image
process 53 image
process 54 image
process 55 image
process 56 image
process 57 image
process 58 image
process 59 image
proces

In [4]:
from torch import nn
import h5py
import numpy as np
from torch.utils.data import Dataset
import torch.optim as optim
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm

class SRCNN(nn.Module):
    def __init__(self, num_channels=1):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=9, padding=9 // 2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=5 // 2)
        self.conv3 = nn.Conv2d(32, num_channels, kernel_size=5, padding=5 // 2)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

class TrainDataset(Dataset):
    def __init__(self, h5_file):
        super(TrainDataset, self).__init__()
        self.h5_file = h5_file

    def __getitem__(self, idx):
        with h5py.File(self.h5_file, 'r') as f:
            return np.expand_dims(f['lr'][idx] / 255., 0), np.expand_dims(f['hr'][idx] / 255., 0)

    def __len__(self):
        with h5py.File(self.h5_file, 'r') as f:
            return len(f['lr'])


# hyperparameters
lr = 1e-4
train_file = "/content/drive/MyDrive/CS17 project/data/train.hdf5"
batch_size = 256
num_workers = 2
num_epochs = 5 

model = SRCNN().cuda()
criterion = nn.MSELoss()
optimizer = optim.Adam([
    {'params': model.conv1.parameters()},
    {'params': model.conv2.parameters()},
    {'params': model.conv3.parameters(), 'lr': lr * 0.1}
], lr=lr)

train_dataset = TrainDataset(train_file)
train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=batch_size,
                              shuffle=True,
                              num_workers=num_workers,
                              pin_memory=True,
                              drop_last=True)


for epoch in range(num_epochs):
    model.train()
    epoch_losses = AverageMeter()

    with tqdm(total=(len(train_dataset) - len(train_dataset) % batch_size)) as t:
        t.set_description('epoch: {}/{}'.format(epoch, num_epochs - 1))

        for data in train_dataloader:
            inputs, labels = data

            inputs = inputs.cuda()
            labels = labels.cuda()

            preds = model(inputs)

            loss = criterion(preds, labels)

            epoch_losses.update(loss.item(), len(inputs))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            t.set_postfix(loss='{:.6f}'.format(epoch_losses.avg))
            t.update(len(inputs))

    model.eval()
    epoch_psnr = AverageMeter()
    epoch_ssim = AverageMeter()

    for data in train_dataloader:
        inputs, labels = data

        inputs = inputs.cuda()
        labels = labels.cuda()

        with torch.no_grad():
            preds = model(inputs).clamp(0.0, 1.0)

        epoch_psnr.update(calc_psnr(preds, labels), len(inputs))

    print('eval psnr: {:.2f}'.format(epoch_psnr.avg))



epoch: 1/4:   0%|          | 0/39936 [00:00<?, ?it/s]

eval psnr: 29.83


epoch: 2/4:   0%|          | 0/39936 [00:00<?, ?it/s]

eval psnr: 31.86


epoch: 3/4:   0%|          | 0/39936 [00:00<?, ?it/s]

eval psnr: 32.46


epoch: 4/4:   0%|          | 0/39936 [00:00<?, ?it/s]

eval psnr: 32.79


epoch: 4/4: 100%|██████████| 39936/39936 [01:35<00:00, 418.93it/s, loss=0.000515]


eval psnr: 33.02


In [18]:
import os
from PIL import Image
from torchvision import transforms

img_dir = "/content/drive/MyDrive/CS17 project/data/HR/"

ssim = 0
listdir=os.listdir(img_dir)
trans = transforms.ToTensor()
for name in listdir:
  image_gt = Image.open(img_dir+name)
  image = image_gt.resize((image_gt.width // scale, image_gt.height // scale), resample=pil_image.BICUBIC)
  blur_image = image.resize((image.width * scale, image.height * scale), resample=pil_image.BICUBIC)
  reconstructed_img = model(trans(blur_image).cuda()).clamp(0.0, 1.0)
  ssim = ssim + compare_ssim(image_gt, reconstructed_img)
print('Average SSIM: {:.2f}'.format(ssim/len(files)))



RuntimeError: ignored